# Business Intelligence and Applications
## Assignment 5: Network Analysis

Zhe Huang, 2020.5

---
### The goal:
> - Analyse and discuss the structure of the network (is there a center; are there any hubs; etc.) and whether the nodes can be divided into meaningful communities and if so, find the central character/s for each community.
> - Find the most important nodes (characters) in the network. List the top 25 characters according to the computed centrality measures.
> - Find the best way to organise and show the network: experiment with layouts for the whole network or for specific subsets of nodes. Please, note, that Gephy and Cytoscape can take some time to apply some of the layouts to the whole subject network, probably due to its size. In such cases, try applying the change of layout to the meaningfully selected subsets of nodes.
> - Reflect the characteristics of the network and the results of the performed analysis on the network’s visualisation by making use of the tools provided by the selected application (e.g. colouring, changing size/thickness, etc.).

---

This notebook explores and analyzes the character interaction network of the HBO’s "Game of Thrones" series (seasons 1-8).


In order to illustrate the interactive graph visualization, Jupyter Notebook provides a tool to load and run the JavaScript. It will fetch the ipynb file from Github.

For this assignment, the link is [here](https://nbviewer.jupyter.org/github/onlyacat/Mathematicians_Network/blob/master/main.ipynb).


In this project I used `Python 3.7` as programming language. I also chose `pyecharts` to draw the interactive pictures and `networkx` to analyze the graph because I used those tools before.

In [48]:
import random
import math
import pandas as pd
import networkx as nx
import prettytable
from pyecharts import options as opts
from pyecharts.charts import Graph, Bar, Line, Radar
from networkx.algorithms import approximation
from networkx.algorithms import community

#### 0. Simple preparations.

- `randomcolor()`: generates a random type color.

- `extract_data()`: read the data from `xlsx` files and then put them into the `networkx` Graph.

In [27]:
def randomcolor():
    colorArr = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F']
    color = ""
    for i in range(6):
        color += colorArr[random.randint(0, 14)]
    return "#" + color

def extract_data():
    edges = pd.read_excel('got_edges.xlsx')
    nodes = pd.read_excel('got_nodes.xlsx')

    G = nx.Graph()

    nodes.apply(lambda x: G.add_node(x.Id, name=x.Label), axis=1)
    edges.apply(lambda x: G.add_edge(x.Source, x.Target, weight=x.Weight), axis=1)

    return G


G = extract_data()

### Question 1

##### Analyse and discuss the structure of the network (is there a center; are there any hubs; etc.) and whether the nodes can be divided into meaningful communities and if so, find the central character/s for each community.
---


- `analyze_basics()`: Use the functions `networkx` provided. 

The graph contains **405** nodes and **2635** edges. The density is **0.032** and the average degree is **13.01**. 

Assortativity measures the similarity of connections in the graph with respect to the node degree. The value of degree assortativity coefficient is **-0.15**, showing that the network is disassortative.


In [4]:
def analyze_basics():
    density = nx.density(G)
    average_degree = sum(dict(nx.degree(G)).values()) / len(dict(nx.degree(G)).values())  # print
    number_of_nodes = len(G.nodes)
    number_of_edges = len(G.edges)
    degree_assortativity = nx.degree_assortativity_coefficient(G)
    table = prettytable.PrettyTable(
        ['Number of Nodes', 'Number Of Edges', 'Density', 'Average Degree', 'Degree Assortativity'])
    table.add_row([number_of_nodes, number_of_edges, density, average_degree, degree_assortativity])
    print(table)

analyze_basics()

+-----------------+-----------------+----------------------+--------------------+----------------------+
| Number of Nodes | Number Of Edges |       Density        |   Average Degree   | Degree Assortativity |
+-----------------+-----------------+----------------------+--------------------+----------------------+
|       405       |       2635      | 0.032208776433198874 | 13.012345679012345 | -0.15069757673076511 |
+-----------------+-----------------+----------------------+--------------------+----------------------+


- `draw_the_whole_graph()`

Here the graph shows the relation of the nodes on this network in **force** layout. It is easy to find that the graph is not connected. It exists a small subgraph, containing three nodes and they are fully connected.

In [13]:
def draw_the_whole_graph():
    nodes = [opts.GraphNode(name=x,value=G.degree[x])for x in G.nodes]
    links = [opts.GraphLink(source=x, target=y, value=G.edges[x, y]['weight']) for x, y in G.edges]

    c = (Graph().add(series_name="",nodes=nodes,links=links,layout='force',is_roam=True,is_focusnode=True,
                     label_opts=opts.LabelOpts(is_show=False),is_draggable=True,)
            .set_global_opts(title_opts=opts.TitleOpts(title="Graph in force view")))
    return c

c = draw_the_whole_graph()

In [14]:
c.render_notebook()

In order to apply for the analysis, first the larger connected component should be extracted from the graph. Therefore I create a subgraph from the original graph and remove the smaller component **{'KEGS', 'BLACK_JACK', 'MULLY'}**.

I will use the max connected subgraph below if the algorithm requires a connected component as an input.

In [15]:
def connected_detection():
    connected = nx.is_connected(G)
    GG = None
    if not connected:
        components = list(nx.connected_components(G))

        table = prettytable.PrettyTable(
            ['connected', 'Nodes Of Smaller Components', 'Smaller Components'])
        table.add_row([connected, min([len(x) for x in components]), components[1]])
        print(table)

        GG = G.subgraph(components[0])
        print(nx.info(GG))

    return GG


GG = connected_detection()

+-----------+-----------------------------+---------------------------------+
| connected | Nodes Of Smaller Components |        Smaller Components       |
+-----------+-----------------------------+---------------------------------+
|   False   |              3              | {'KEGS', 'BLACK_JACK', 'MULLY'} |
+-----------+-----------------------------+---------------------------------+
Name: 
Type: Graph
Number of nodes: 402
Number of edges: 2632
Average degree:  13.0945


- `analyze_clustering()`: Use the functions `networkx` provided. 

The `Average clustering` and `Average clustering coefficient` denote the willing that the nodes tend to cluster together or not. 

`Average shortest path length` means the average number of steps along the shortest paths for all possible pairs of network nodes. It can measure the efficiency of information on a network. The value is **2.682**, showing that for every two nodes on this graph, it takes about 2.7 edges to reach.

There is a positive correlation between the `Efficiency ` and the `Shortest path length`. The average local efficiency is the average of the local efficiencies of each node and the average global efficiency of a graph is the average efficiency of all pairs of nodes.

In [16]:
def analyze_clustering(G):
    average_clustering_coefficient = approximation.average_clustering(G)
    average_clustering = nx.average_clustering(G)
    average_shortest_path_length = nx.average_shortest_path_length(G)
    local_efficiency = nx.local_efficiency(G)
    global_efficiency = nx.global_efficiency(G)
    table = prettytable.PrettyTable(
        ['Average clustering', 'Average clustering coefficient', 'Average shortest path length'])
    table.add_row([average_clustering, average_clustering_coefficient, average_shortest_path_length])
    print(table)
    table = prettytable.PrettyTable(['Local efficiency', 'Global efficiency'])
    table.add_row([local_efficiency, global_efficiency])
    print(table)

analyze_clustering(GG)

+--------------------+--------------------------------+------------------------------+
| Average clustering | Average clustering coefficient | Average shortest path length |
+--------------------+--------------------------------+------------------------------+
| 0.6578481764404378 |             0.644              |      2.6824729221721815      |
+--------------------+--------------------------------+------------------------------+
+--------------------+---------------------+
|  Local efficiency  |  Global efficiency  |
+--------------------+---------------------+
| 0.7599994731106516 | 0.40530038916030003 |
+--------------------+---------------------+


The `center` is the node with eccentricity equal to radius. Obviously, **BRAN, BRIENNE, CATELYN, CERSEI** are the centers and **TYRION** is the barycenter on this graph. The `diameter` is the maximum eccentricity, showing that two farthest nodes have the distance of 6 on this graph.

In [19]:
def analyze_distance(G):
    center = nx.center(G)
    center = center if len(center) <= 5 else center[:4]
    barycenter = nx.barycenter(G)
    diameter = nx.diameter(G)
    table = prettytable.PrettyTable(['center', 'barycenter', 'diameter'])
    table.add_row([center, barycenter, diameter])
    print(table)


analyze_distance(GG)

+------------------------------------------+------------+----------+
|                  center                  | barycenter | diameter |
+------------------------------------------+------------+----------+
| ['BRAN', 'BRIENNE', 'CATELYN', 'CERSEI'] | ['TYRION'] |    6     |
+------------------------------------------+------------+----------+


- `calculate_community()`: Use the functions `networkx` provided. 

Here the algorithm `asyn_lpa_communities` will find the most frequently among that nodes neighbors after initializing each node with a unique label. Therefore it will generate different communities randomly. I choose the size of 8 so that it can be illustrated in the graph. Then it prints the length of each communities and the first 10 elements.

In [77]:
def calculate_community():
    while True:
        cate = list(community.asyn_lpa_communities(G, 'weight'))
        if len(cate) == 8:
            break
    
    cate = [list(x) for x in cate]
    for index,x in enumerate(cate):
        print(str(len(x))+' elements. '+str(x[:10]))
        for y in x:
            G.nodes[y]['cate'] = index
        
calculate_community()

179 elements. ['HIGH_SPARROW', 'MERRY', 'LOTHAR', 'WAYMAR_ROYCE', 'MARTYN', 'QYBURN', 'NYMERIA', 'VARDIS_EGEN', 'MAESTER_WOLKAN', 'EDMURE']
23 elements. ['CAPTAINS_DAUGHTER', 'HARRAG', 'TANSY', 'TORTURER', 'WINTERFELL_SHEPHERD', 'DAGMER', 'DROWNED_PRIEST', 'WALDA', 'RALF', 'BLACK_LORREN']
86 elements. ['KARL_TANNER', 'MARYA', 'CRASTER', 'GILLY', 'MAG_THE_MIGHTY', 'NIGHT_KING', 'FARMER', 'MANDERLY', 'OLLY', 'YGRITTE']
43 elements. ['BITER', 'VISENYA', 'OWEN', 'SALLY', 'TERNESIO_TERYS', 'FLYNN', 'RORGE', 'GATINS', 'HOT_PIE', 'INNKEEPERS_DAUGHTER']
55 elements. ['GREY_WORM', 'MANSERVANT', 'RAZDAL', 'MEEREEN_CHAMPION', 'QUICK', 'RAKHARO', 'IRRI', 'BELICHO', 'MOSSADOR', 'PYATT_PREE']
13 elements. ['MEERA', 'BILLY', 'BRAN', 'OLD_NAN', 'RICKON', 'PORTAN', 'THREE_EYED_RAVEN', 'FARLEN', 'LEAF', 'JOJEN']
3 elements. ['KEGS', 'BLACK_JACK', 'MULLY']
3 elements. ['CRAYAH', 'DIRAH', 'WILLIAM']


# Question 2

##### Find the most important nodes (characters) in the network. List the top 25 characters according to the computed centrality measures.
---

- `draw_centrality()`
Here the radar graph illustrates the distribution of centralities. Degree centrality，betweenness centrality and closeness centrality are three concepts to measure the node centrality.

Degree centrality is measured by the number of edges connected to this node.

Betweenness centrality is measured by the mentioned number in the shortest paths. As we all know the nodes rely on the shortest path to communicate. If a node always appear in the shortest path of other nodes, it has a high betweenness centrality.

Closeness centrality is measured by the shortest distance from this node to other nodes. If a node can easily reach other nodes without a long path, it can be considered as the centre of the graph and will have a high closeness centrality.

In the function first it calculates the three centralities for each node and then only keep the top 30 nodes for each centrality. Then it intersects three sets and chooses the common nodes.

Then it prints the **TOP 25** characters with highest total centrality. I just do a simple sum here to calculate the centrality. Then it shows the **TOP 10** characters for three centralities in detail.


In [43]:
def draw_centrality():
    degree_centrality = nx.degree_centrality(G)
    closeness_centrality = nx.closeness_centrality(G)
    betweenness_centrality = nx.betweenness_centrality(G)
    degree_centrality_sort = sorted(degree_centrality, key=lambda x: degree_centrality[x], reverse=True)
    closeness_centrality_sort = sorted(closeness_centrality, key=lambda x: closeness_centrality[x], reverse=True)
    betweenness_centrality_sort = sorted(betweenness_centrality, key=lambda x: betweenness_centrality[x], reverse=True)
    top_authors = sorted(
        [(x, degree_centrality[x] + closeness_centrality[x] + betweenness_centrality[x]) for x in set.intersection(
            set(betweenness_centrality_sort[:30]), set(closeness_centrality_sort[:30]),
            set(degree_centrality_sort[:30]))], key=lambda x: x[1], reverse=True)

    table = prettytable.PrettyTable(
        ['Rank', 'Name', 'Degree centrality', 'Closeness centrality', 'Betweenness centrality', 'Sum centrality'])

    [table.add_row(
        [index + 1, value[0], round(degree_centrality[value[0]], 3), round(closeness_centrality[value[0]], 3),
         round(betweenness_centrality[value[0]], 3), round(value[1], 3)]) for index, value in enumerate(top_authors)]

    print(table)

    data = [[G.nodes[x[0]]["name"], [round(degree_centrality[x[0]], 3), round(closeness_centrality[x[0]], 3),
         round(betweenness_centrality[x[0]], 3)]] for x in top_authors[:10]]

    c = (
        Radar()
            .add_schema(schema=[
                opts.RadarIndicatorItem(name="Betweenness centrality [0, 0.4]", max_=0.4, min_=0),
                opts.RadarIndicatorItem(name="Closeness centrality [0, 0.6]", max_=0.6, min_=0),
                opts.RadarIndicatorItem(name="Degree centrality [0, 0.2]", max_=0.2, min_=0),],
            shape="circle",center=["50%", "50%"],radius="80%",splitarea_opt=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)),
            textstyle_opts=opts.TextStyleOpts(color="#000"),
        ).set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )

    for x in data:
        color = randomcolor()
        c.add(series_name=x[0],data=[x[1]],areastyle_opts=opts.AreaStyleOpts(opacity=0.1, color=color),
            linestyle_opts=opts.LineStyleOpts(width=1, color=color),label_opts=opts.LabelOpts(is_show=False))

    return c


c = draw_centrality()

+------+--------------+-------------------+----------------------+------------------------+----------------+
| Rank |     Name     | Degree centrality | Closeness centrality | Betweenness centrality | Sum centrality |
+------+--------------+-------------------+----------------------+------------------------+----------------+
|  1   |    TYRION    |       0.317       |        0.575         |         0.132          |     1.024      |
|  2   |     JON      |       0.257       |        0.549         |         0.091          |     0.898      |
|  3   |     ARYA     |        0.24       |        0.538         |         0.102          |      0.88      |
|  4   |   DAENERYS   |        0.23       |        0.532         |         0.113          |     0.875      |
|  5   |    SANSA     |        0.25       |        0.549         |         0.057          |     0.856      |
|  6   |    JAIME     |       0.223       |        0.539         |         0.055          |     0.817      |
|  7   |    CERSEI 

In [42]:
c.render_notebook()

# Question 3 & 4

##### Find the best way to organise and show the network: experiment with layouts for the whole network or for specific subsets of nodes. Please, note, that Gephy and Cytoscape can take some time to apply some of the layouts to the whole subject network, probably due to its size. In such cases, try applying the change of layout to the meaningfully selected subsets of nodes.

##### Reflect the characteristics of the network and the results of the performed analysis on the network’s visualisation by making use of the tools provided by the selected application (e.g. colouring, changing size/thickness, etc.).
---

Here I use `force` and `circular` two types of layout to show the graph. 

Also the colour represents the community of the node, the size represents the node's degree and the edge thickness represents the weight.

The formula of the size and the edge thickess is: $ceil(\frac x {max}*10)*3)$. So that the degree and the weight will be mapped into [3,30].

In [78]:
max_deg = max([x[1] for x in G.degree])
max_edge = max([G.edges[x]['weight'] for x in G.edges])

def draw_force_graph():
    nodes = [opts.GraphNode(name=x,value=G.degree[x],
                            symbol_size=math.ceil(G.degree[x]/max_deg*10)*3,
                            category=G.nodes[x]['cate']) for x in G.nodes]
    links = [opts.GraphLink(source=x, target=y, value=G.edges[x, y]['weight'],linestyle_opts=opts.LineStyleOpts(width=math.ceil(G.edges[x, y]['weight']*10 / max_edge)*2)) for x, y in G.edges]
    categories = [{'name': str(x+1)} for x in range(8)]
    
    c = (Graph().add(series_name="",nodes=nodes,links=links,layout='force',is_roam=True,is_focusnode=True,categories=categories,
                     label_opts=opts.LabelOpts(is_show=False),is_draggable=True,)
            .set_global_opts(title_opts=opts.TitleOpts(title="Graph in force view")))
    return c

c = draw_force_graph()

In [79]:
c.render_notebook()

In [80]:
def draw_circular_graph():
    nodes = [opts.GraphNode(name=x,value=G.degree[x],
                            symbol_size=math.ceil(G.degree[x]/max_deg*10)*3,
                            category=G.nodes[x]['cate']) for x in G.nodes]
    links = [opts.GraphLink(source=x, target=y, value=G.edges[x, y]['weight'],linestyle_opts=opts.LineStyleOpts(width=math.ceil(G.edges[x, y]['weight']*10 / max_edge)*2)) for x, y in G.edges]
    categories = [{'name': str(x+1)} for x in range(8)]
    
    c = (Graph().add(series_name="",nodes=nodes,links=links,layout='circular',is_roam=True,is_focusnode=True,categories=categories,
                     label_opts=opts.LabelOpts(is_show=False),is_draggable=True,)
            .set_global_opts(title_opts=opts.TitleOpts(title="Graph in force view")))
    return c

c = draw_circular_graph()

In [81]:
c.render_notebook()